# Effective music playing using reinforcement learning


Data: https://www.kaggle.com/code/karnikakapoor/music-generation-lstm/


Original data site: http://www.piano-midi.de



Data is in the data folder

In [24]:
#Importing Libraries
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio

from sklearn.model_selection import train_test_split
import pathlib
import music21
from music21 import *

import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
import sys
import os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

In [101]:
#Loading the list of chopin's midi files as stream 
filepath = os.getcwd()+"/data/debussy/"
#Getting midi files
all_midis= []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        tr = filepath+i
        midi = converter.parse(tr)
        all_midis.append(midi)

In [124]:
#Helping function        
def extract_notes(files):
    notes = [0]*len(files)
    pick = None
    for idx, file in enumerate(files):
        notes[idx]=[]
        songs = instrument.partitionByInstrument(file)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes[idx].append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    #notes[idx].append(".".join(str(n) for n in element.normalOrder))
                    notes[idx].append(".".join(str(note.Note(idx).pitch) for idx in element.normalOrder))

    return notes
#Getting the list of notes as Corpus
Corpus= extract_notes(all_midis)
print("Total songs in all the Chopin midis in the dataset:", len(Corpus))

Total songs in all the Chopin midis in the dataset: 7


In [137]:
chord.tables.tnIndexToChordInfo

{(1, 1, 0): {'name': ('unison', 'monad', 'singleton')},
 (2,
  1,
  0): {'name': ('interval class 1',
   'minor second',
   'm2',
   'half step',
   'semitone')},
 (2,
  2,
  0): {'name': ('interval class 2',
   'major second',
   'M2',
   'whole step',
   'whole tone')},
 (2, 3, 0): {'name': ('interval class 3', 'minor third', 'm3')},
 (2, 4, 0): {'name': ('interval class 4', 'major third', 'M3')},
 (2, 5, 0): {'name': ('interval class 5', 'perfect fourth', 'P4')},
 (2, 6, 0): {'name': ('tritone', 'diminished fifth', 'augmented fourth')},
 (3, 1, 0): {'name': ('chromatic trimirror',)},
 (3, 2, 1): {'name': ('phrygian trichord',)},
 (3, 2, -1): {'name': ('minor trichord',)},
 (3, 3, 1): {'name': ('major-minor trichord',)},
 (3, 3, -1): {'name': ('major-minor trichord',)},
 (3, 4, 1): {'name': ('incomplete major-seventh chord',)},
 (3, 4, -1): {'name': ('incomplete major-seventh chord',)},
 (3, 5, 1): {'name': ('tritone-fourth',)},
 (3, 5, -1): {'name': ('tritone-fourth',)},
 (3, 6, 0):

In [62]:
#First Lets write some functions that we need to look into the data
def show(music):
    display(Image(str(music.write("lily.png"))))
    
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
            chord_snip = chord.Chord(notes)
            chord_snip.offset = offset
            Melody.append(chord_snip)
        # pattern is a note
        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

Melody_Snippet = chords_n_notes(Corpus[0])
show(Melody_Snippet)

Fontconfig warning: ignoring UTF-8: not a valid region tag
Changing working directory to: `/var/folders/p0/2qnwrmjn7yd0ks7pnlb3d7mc0000gn/T/music21'
Processing `/var/folders/p0/2qnwrmjn7yd0ks7pnlb3d7mc0000gn/T/music21/tmpk70o8nq2.ly'
Parsing...
/var/folders/p0/2qnwrmjn7yd0ks7pnlb3d7mc0000gn/T/music21/tmpk70o8nq2.ly:1325:5: error: unknown escaped string: `\RemoveEmptyStaffContext'
    
    \RemoveEmptyStaffContext
/var/folders/p0/2qnwrmjn7yd0ks7pnlb3d7mc0000gn/T/music21/tmpk70o8nq2.ly:1326:5: error: syntax error, unexpected \override, expecting '='
    
    \override VerticalAxisGroup #'remove-first = ##t
/var/folders/p0/2qnwrmjn7yd0ks7pnlb3d7mc0000gn/T/music21/tmpk70o8nq2.ly:1326:33: warning: deprecated: missing `.' in property path VerticalAxisGroup.remove-first
    \override VerticalAxisGroup 
                                #'remove-first = ##t
/var/folders/p0/2qnwrmjn7yd0ks7pnlb3d7mc0000gn/T/music21/tmpk70o8nq2.ly:1328:2: error: syntax error, unexpected '}'
 
 }
/var/folders/p0/2qn

LilyTranslateException: cannot find tmpk70o8nq2.ly.png or the full path /var/folders/p0/2qnwrmjn7yd0ks7pnlb3d7mc0000gn/T/music21/tmpk70o8nq2.ly.png original file was /var/folders/p0/2qnwrmjn7yd0ks7pnlb3d7mc0000gn/T/music21/tmpk70o8nq2.ly

In [223]:
note.Note(21).nameWithOctave


'A0'

In [227]:
note.Note('A0').pitch.midi

21